<a href="https://www.kaggle.com/code/kangnurrohman/polarity-determination-3-algorithm-nb-rf-svm?scriptVersionId=112064333" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/review-of-the-application-pln-mobile/review of the application PLN mobile.xlsx


## Install library

In [19]:
!pip install sastrawi
!pip install contractions

import warnings
warnings.filterwarnings('ignore')

## Load Library

In [22]:
import re
import tqdm
import nltk
import string
import unicodedata
import contractions

from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## Load Data

In [4]:
df = pd.read_excel('/kaggle/input/review-of-the-application-pln-mobile/review of the application PLN mobile.xlsx')
df.rename(columns = {'content':'review', 'score':'sentiment'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201042 entries, 0 to 201041
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              201042 non-null  object        
 1   userName              201042 non-null  object        
 2   userImage             201042 non-null  object        
 3   review                201042 non-null  object        
 4   sentiment             201042 non-null  int64         
 5   thumbsUpCount         201042 non-null  int64         
 6   reviewCreatedVersion  174363 non-null  object        
 7   at                    201042 non-null  datetime64[ns]
 8   replyContent          195639 non-null  object        
 9   repliedAt             195639 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 15.3+ MB


In [5]:
df = df.sort_values(by='at', ascending=False)
df = df[['review', 'sentiment']]
df.head()

,review,sentiment
0,Sangat bermanfaat👍👍,5
1,PLN terbaik,5
2,😀,5
3,Mudah dan simpel untuk menikmati pelayanan PLN...,5
4,Baik,5


## Exploratory Data Analysis

In [5]:
df = df.replace({'sentiment' : {1:'negative', 2:'negative', 3:'neutral', 4:'positive', 5: 'positive' }})
df.head()

,reviewId,userName,userImage,review,sentiment,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,7b2e334b-5f37-482e-b145-9ac8d77255ab,Lisa Mananoma,https://play-lh.googleusercontent.com/a-/ACNPE...,Sangat bermanfaat👍👍,positive,0,5.2.31,2022-11-01 02:30:56,NaN,NaT
1,3e4c6709-383a-4dfe-b6f7-5caa0082de2a,Seni Nurjiah,https://play-lh.googleusercontent.com/a-/ACNPE...,PLN terbaik,positive,0,5.2.25,2022-11-01 02:30:44,"Hello Kak Nurjiah, makasih yaa buat review & r...",2022-10-30 05:35:57
2,d98b38bf-e935-4b38-8eca-f8f51f4543aa,koko andika,https://play-lh.googleusercontent.com/a/ALm5wu...,😀,positive,0,NaN,2022-11-01 02:28:40,NaN,NaT
3,a9ada1b6-215c-4f11-b8ec-90fe46604724,Hasnur ACHYAR Sungkar,https://play-lh.googleusercontent.com/a-/ACNPE...,Mudah dan simpel untuk menikmati pelayanan PLN...,positive,0,5.2.30,2022-11-01 02:28:27,Trims yah Kak Hasnur buat ulasan dan bintangny...,2022-01-09 16:46:43
4,7b571d6f-b484-424b-be98-19a4f9ef59c9,Putu Nanda,https://play-lh.googleusercontent.com/a/ALm5wu...,Baik,positive,0,5.2.30,2022-11-01 02:27:49,NaN,NaT


In [6]:
df.sentiment.value_counts()

positive    175750
negative     19925
neutral       5367
Name: sentiment, dtype: int64

In [7]:
#removing neutral
df = df[df.sentiment != "neutral"]

## Data Preprocessing

In [16]:
def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def stopwords_removal(words):
    list_stopwords = nltk.corpus.stopwords.words('indonesian')
    #list_stopwords = stopwords.words('indonesian')
    #list_stopwords.extend([])
    #txt_stopword = pd.read_csv("#", names= ["stopwords"], header = None)
    #list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
    return [word for word in words if word not in list_stopwords]

def stemmed_wrapper(term):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(term)

In [24]:
def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    doc = word_tokenize(doc)
    doc = stopwords_removal(doc)
    #doc = stemmed_wrapper(doc)
    norm_docs.append(doc)

  norm_docs = [" ".join(word) for word in norm_docs]
  return norm_docs

In [25]:
%%time
df.review = pre_process_corpus(df.review)

100%|██████████| 195675/195675 [01:53<00:00, 1719.84it/s]


CPU times: user 1min 50s, sys: 3.9 s, total: 1min 54s
Wall time: 1min 53s


## TF-IDF

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df['review'].astype('U'))
text_tf

<195675x51117 sparse matrix of type '<class 'numpy.float64'>'
	with 827030 stored elements in Compressed Sparse Row format>

## Splitting Data

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_tf, df['sentiment'], test_size=0.1, random_state = 42)

## Handle Data Imbalance (Oversampling)

In [28]:
#OVERSAMPLING

print("Before OverSampling, counts of label 'Positive': {}".format(sum(y_train == 'positive')))
print("Before OverSampling, counts of label 'Negative': {}".format(sum(y_train == 'negative')))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label 'positive': {}".format(sum(y_train_res == 'positive')))
print("After OverSampling, counts of label 'negative': {}".format(sum(y_train_res == 'negative')))

Before OverSampling, counts of label 'Positive': 158160
Before OverSampling, counts of label 'Negative': 17947
After OverSampling, the shape of train_X: (316320, 51117)
After OverSampling, the shape of train_y: (316320,) 

After OverSampling, counts of label 'positive': 158160
After OverSampling, counts of label 'negative': 158160


## Naive Bayes

In [29]:
from sklearn.naive_bayes import MultinomialNB  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("MultinomialNB Accuracy:" , accuracy_score(y_test,predicted))
print("MultinomialNB Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("MultinomialNB Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("MultinomialNB f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))                  

MultinomialNB Accuracy: 0.8827677841373671
MultinomialNB Precision: 0.7251774682933378
MultinomialNB Recall: 0.9004661279762436
MultinomialNB f1_score: 0.772475781837197
confusion matrix:
 [[ 1825   153]
 [ 2141 15449]]

              precision    recall  f1-score   support

    negative       0.46      0.92      0.61      1978
    positive       0.99      0.88      0.93     17590

    accuracy                           0.88     19568
   macro avg       0.73      0.90      0.77     19568
weighted avg       0.94      0.88      0.90     19568



## SVM

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='linear').fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("Support Vector Machine Accuracy:" , accuracy_score(y_test,predicted))
print("Support Vector Machine Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("Support Vector Machine Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("Support Vector Machine f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))  

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("Random Forest Accuracy:" , accuracy_score(y_test,predicted))
print("Random Forest Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("Random Forest Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("Random Forest f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))  